In [64]:
import os
import pandas as pd
import sql as gd

In [80]:
# flights_faso = gd.get_data('SELECT * FROM flights_faso')
# flights_faso.head()
flights_fs = gd.get_data('SELECT * FROM flights_fs')
flights_fs

/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,distance,cancelled,diverted
0,2012-10-01,933.0,900,33.0,1305.0,1205,60.0,AA,N320AA,1,JFK,LAX,317.0,2475.0,0,0
1,2012-10-02,906.0,900,6.0,1200.0,1205,-5.0,AA,N319AA,1,JFK,LAX,328.0,2475.0,0,0
2,2012-10-03,900.0,900,0.0,1225.0,1205,20.0,AA,N319AA,1,JFK,LAX,340.0,2475.0,0,0
3,2012-10-04,853.0,900,-7.0,1205.0,1205,0.0,AA,N325AA,1,JFK,LAX,336.0,2475.0,0,0
4,2012-10-05,853.0,900,-7.0,1236.0,1205,31.0,AA,N325AA,1,JFK,LAX,368.0,2475.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325773,2012-11-10,824.0,824,0.0,1346.0,1220,NaN,YV,N518LR,3733,ELP,ORD,NaN,1236.0,0,1
325774,2012-11-10,1045.0,1049,-4.0,1433.0,1216,NaN,YV,N522LR,3734,MCI,ORD,NaN,403.0,0,1
325775,2012-11-23,834.0,840,-6.0,1029.0,900,NaN,YV,N922FJ,2819,PHX,LGB,NaN,355.0,0,1
325776,2012-11-27,1511.0,1515,-4.0,1753.0,1643,NaN,YV,N909FJ,2744,AUS,PHX,NaN,872.0,0,1


In [76]:
# getting lat and lon of selected airports from airports table in database

points = gd.get_data("SELECT faa, lat,lon FROM airports WHERE faa in ('JFK', 'IAD', 'LGA', 'DCA', 'PHX', 'ORD', 'LAX')")
points
         


/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,faa,lat,lon
0,PHX,33.434299,-112.012001
1,LAX,33.942501,-118.407997
2,DCA,38.852100,-77.037697
3,LGA,40.777199,-73.872597
4,IAD,38.944500,-77.455803
5,JFK,40.639801,-73.778900
6,ORD,41.978600,-87.904800


In [46]:
import requests

url = "https://meteostat.p.rapidapi.com/point/daily"

querystring = {"lat": "40.77719879","lon": "-73.87259674","start":"2012-10-29","end":"2012-10-29"}
# https://meteostat.p.rapidapi.com/point/daily?lat=43.6667&lon=-79.4&start=2020-01-01&end=2020-01-31&alt=184'
headers = {
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com",
	"X-RapidAPI-Key": ""
}
r = requests.request("GET", url, headers=headers, params=querystring)

print(r.text)
type(r)

{"meta":{"generated": "2022-05-06 08:32:48", "stations": ["KNYC0", "72502", "KTEB0", "74486"]},"data":[{"date":"2012-10-29","tavg":14.5,"tmin":12.2,"tmax":17.8,"prcp":15.2,"snow":0.0,"wdir":35.0,"wspd":54.7,"wpgt":null,"pres":null,"tsun":null}]}


requests.models.Response

In [47]:
import json

print(json.dumps(r.json(), indent=2))

{
  "meta": {
    "generated": "2022-05-06 08:32:48",
    "stations": [
      "KNYC0",
      "72502",
      "KTEB0",
      "74486"
    ]
  },
  "data": [
    {
      "date": "2012-10-29",
      "tavg": 14.5,
      "tmin": 12.2,
      "tmax": 17.8,
      "prcp": 15.2,
      "snow": 0.0,
      "wdir": 35.0,
      "wspd": 54.7,
      "wpgt": null,
      "pres": null,
      "tsun": null
    }
  ]
}


In [116]:
# Set the url
url = "https://meteostat.p.rapidapi.com/point/daily"

# Set locations

points = {"DCA": ["38.852100", "-77.037697"],
	     "LGA": ["40.777199", "-73.872597"],
	     "IAD":	["38.944500", "-77.455803"],
	     "JFK":	["40.639801", "-73.778900"],
         "MDW":	["41.785999", "-87.752403"],
         "ORD":	["41.9786",	"-87.9048"],
         "LAX":	["33.94250107",	"-118.4079971"],
         "PHX":	["33.43429946899414", "-112.01200103759766"]}


# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])
# Loop through all locations
for key, value in points.items():
    # Create final url
    # Request data from url
    querystring = {"lat":value[0], "lon":value[1], "start":"2012-10-01", "end":"2012-11-29"}
    # https://meteostat.p.rapidapi.com/point/daily?lat=43.6667&lon=-79.4&start=2020-01-01&end=2020-01-31&alt=184'
    headers = {
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com",
    "X-RapidAPI-Key": "06cc1bc58bmsh6b0e697965add32p1e5320jsnf3103b779dfa"
    }
    r = requests.request("GET", url, headers=headers, params=querystring)

    if r.status_code != 200:
        print(r.json())
        break
    
    import time 
    time.sleep(0.5) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data", 
                                        errors='ignore')

    
    weather_temp_df['location'] = [key] * 60
    
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,location
0,2012-10-01,16.9,12.2,22.2,0.0,0.0,NaN,5.8,None,1014.0,None,DCA
1,2012-10-02,20.3,18.3,22.2,10.9,0.0,NaN,7.9,None,1013.2,None,DCA
2,2012-10-03,23.8,21.1,27.8,0.0,0.0,NaN,4.7,None,1017.6,None,DCA
3,2012-10-04,23.9,18.9,27.2,0.3,0.0,NaN,7.6,None,1019.9,None,DCA
4,2012-10-05,21.8,16.1,28.3,0.0,0.0,NaN,7.9,None,1018.7,None,DCA
...,...,...,...,...,...,...,...,...,...,...,...,...
475,2012-11-25,20.1,13.9,27.8,0.0,None,NaN,5.0,None,1011.1,None,PHX
476,2012-11-26,18.8,12.8,26.7,0.0,None,NaN,6.1,None,1014.5,None,PHX
477,2012-11-27,19.9,12.8,28.9,0.0,None,NaN,5.8,None,1017.8,None,PHX
478,2012-11-28,19.9,13.9,26.7,0.0,None,NaN,6.1,None,1017.6,None,PHX


In [119]:

flights_selected = flights_fs[['flight_date', 'dep_delay', 'origin', 'dest', 'cancelled', 'diverted']]
weather_df_selected = weather_df[['date', 'tavg', 'tmin', 'tmax', 'wspd', 'location']]



In [ ]:
weather_df_selected['date'] = weather_df_selected['date'].map(pd.to_datetime)
weather_df_selected.info()

In [215]:
all_origin = flights_selected.merge(weather_df_selected, 
    left_on=['flight_date', 'origin'], right_on=['date', 'location'])
all_origin
# list_dates = ["2012-10-22","2012-10-23", "2012-10-24", "2012-10-25", "2012-10-26", "2012-10-27",  "2012-10-28", "2012-10-29", "2012-10-30", "2012-10-31", "2012-11-01", "2012-11-02"]
all_hit =all_origin.query('origin in ["JFK","IAD","LGA","DCA"] and date in ["2012-10-22","2012-10-23", "2012-10-24", "2012-10-25", "2012-10-26", "2012-10-27",  "2012-10-28", "2012-10-29", "2012-10-30", "2012-10-31", "2012-11-01", "2012-11-02"]')

In [207]:
# attemps with Sergio 
# flights_selected.origin.unique().tolist().isin(weather_df_selected.location.unique())
# type(flights_selected.origin.unique().tolist())
# pd.Series(flights_selected.origin.unique()).isin(weather_df_selected.location.unique())
# all_origin2.dropna(inplace=True)
# all_origin2.info()
# all_origin2

,flight_date,dep_delay,origin,dest,cancelled,diverted,date,tavg,tmin,tmax,wspd,location
0,2012-10-01,33.0,JFK,LAX,0,0,2012-10-01,17.1,12.2,22.2,18.4,JFK
1,2012-10-01,-5.0,JFK,LAX,0,0,2012-10-01,17.1,12.2,22.2,18.4,JFK
2,2012-10-01,-5.0,JFK,LAX,0,0,2012-10-01,17.1,12.2,22.2,18.4,JFK
3,2012-10-01,-7.0,JFK,LAX,0,0,2012-10-01,17.1,12.2,22.2,18.4,JFK
4,2012-10-01,-3.0,JFK,SFO,0,0,2012-10-01,17.1,12.2,22.2,18.4,JFK
...,...,...,...,...,...,...,...,...,...,...,...,...
325773,2012-11-30,8.0,MDW,PHX,0,0,NaT,NaN,NaN,NaN,NaN,NaN
325774,2012-11-30,29.0,MDW,PHX,0,0,NaT,NaN,NaN,NaN,NaN,NaN
325775,2012-11-30,0.0,MDW,PHX,0,0,NaT,NaN,NaN,NaN,NaN,NaN
325776,2012-11-30,-1.0,MDW,PHX,0,0,NaT,NaN,NaN,NaN,NaN,NaN


In [216]:
import matplotlib.pyplot as plt
import seaborn as sns
# sns.boxplot(total['truck_bday'])
# plt.scatter(x='dep_delay', y='wspd')
# all_hit2.plot(kind='scatter', x='dep_delay', y='wspd');
# X3 = all_hit[['dep_delay', 'wspd', 'cancelled', 'tavg', 'diverted' ]]
# sns.pairplot(X3);
all_hit.corr()

,dep_delay,cancelled,diverted,tavg,tmin,tmax,wspd
dep_delay,1.000000,-0.005168,0.031778,-0.044133,-0.037353,-0.045912,0.067349
cancelled,-0.005168,1.000000,-0.017904,-0.436782,-0.301371,-0.448686,0.660052
diverted,0.031778,-0.017904,1.000000,0.019125,0.013251,0.018148,-0.016050
tavg,-0.044133,-0.436782,0.019125,1.000000,0.881334,0.910928,-0.299503
tmin,-0.037353,-0.301371,0.013251,0.881334,1.000000,0.631656,-0.163386
tmax,-0.045912,-0.448686,0.018148,0.910928,0.631656,1.000000,-0.370052
wspd,0.067349,0.660052,-0.016050,-0.299503,-0.163386,-0.370052,1.000000


In [217]:
all_origin = pd.merge(flights_selected,weather_df_selected, 
    left_on=['flight_date', 'origin'], right_on=['date', 'location'])
# list_dates = ["2012-10-22","2012-10-23", "2012-10-24", "2012-10-25", "2012-10-26", "2012-10-27",  "2012-10-28", "2012-10-29", "2012-10-30", "2012-10-31", "2012-11-01", "2012-11-02"]
all_not_hit =all_origin.query('origin in ["MDW","ORD","LAX","PHX"] and date in ["2012-10-22","2012-10-23", "2012-10-24", "2012-10-25", "2012-10-26", "2012-10-27",  "2012-10-28", "2012-10-29", "2012-10-30", "2012-10-31", "2012-11-01", "2012-11-02"]')

In [218]:
all_not_hit.corr()

,dep_delay,cancelled,diverted,tavg,tmin,tmax,wspd
dep_delay,1.000000,0.015033,0.012827,-0.047319,-0.051555,-0.063770,0.074604
cancelled,0.015033,1.000000,-0.008746,-0.130934,-0.114572,-0.125933,0.128303
diverted,0.012827,-0.008746,1.000000,-0.001714,-0.002582,-0.001648,0.008060
tavg,-0.047319,-0.130934,-0.001714,1.000000,0.945724,0.980058,-0.591379
tmin,-0.051555,-0.114572,-0.002582,0.945724,1.000000,0.884603,-0.603089
tmax,-0.063770,-0.125933,-0.001648,0.980058,0.884603,1.000000,-0.603315
wspd,0.074604,0.128303,0.008060,-0.591379,-0.603089,-0.603315,1.000000


In [132]:
all_dest = pd.merge(flights_selected,weather_df_selected, 
    left_on=['flight_date', 'dest'], right_on=['date', 'location'])
all_dest

,flight_date,dep_delay,origin,dest,cancelled,diverted,date,tavg,tmin,tmax,wspd,location
0,2012-10-01,33.0,JFK,LAX,0,0,2012-10-01,23.2,18.3,28.3,9.0,LAX
1,2012-10-01,-5.0,JFK,LAX,0,0,2012-10-01,23.2,18.3,28.3,9.0,LAX
2,2012-10-01,-2.0,OGG,LAX,0,0,2012-10-01,23.2,18.3,28.3,9.0,LAX
3,2012-10-01,-5.0,JFK,LAX,0,0,2012-10-01,23.2,18.3,28.3,9.0,LAX
4,2012-10-01,-7.0,JFK,LAX,0,0,2012-10-01,23.2,18.3,28.3,9.0,LAX
...,...,...,...,...,...,...,...,...,...,...,...,...
173752,2012-11-29,2.0,PHX,MDW,0,0,2012-11-29,5.8,0.6,11.1,13.9,MDW
173753,2012-11-29,5.0,PHX,MDW,0,0,2012-11-29,5.8,0.6,11.1,13.9,MDW
173754,2012-11-29,4.0,PHX,MDW,0,0,2012-11-29,5.8,0.6,11.1,13.9,MDW
173755,2012-11-29,25.0,PHX,MDW,0,0,2012-11-29,5.8,0.6,11.1,13.9,MDW


In [135]:
from sql import engine
table_name = 'weather_df_fs'

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        weather_df_selected.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        # schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The weather_df_fs table was imported successfully.
